# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

Загружаем данные о городском населении стран

In [ ]:
population_data = pd.read_csv('/kaggle/input/population-by-country-2020/population_by_country_2020.csv')
population_data['urban_pct'] = population_data['Urban Pop %'].str.strip('%').str.replace('N.A.','100').fillna(100).apply(lambda p: float(p) / 100)
population_data['urban_population'] = round(population_data['Population (2020)'] * population_data['urban_pct'])
population_data = population_data[['Country (or dependency)', 'urban_population']]
population_data.columns = ['Country', 'urban_population']
population_data.sample(5)

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

Заполнение NA значений в колонках

In [ ]:
data.fillna({'Reviews': '[]','Cuisine Style': "[]", 'Number of Reviews': 0, 'Price Range': 'unknown'}, inplace=True)

Посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

К категориальным признакам отнесем City и Cuisine Style.

Смотрим в каких городах были отзывы:

In [ ]:
data['City'].unique()

Исправляем Oporto на Porto

In [ ]:
data['City'] = data.City.str.replace('Oporto', 'Porto')

Добавляем методы трансформации и порождения признаков:

In [ ]:
from datetime import datetime
from ast import literal_eval
literal_string_pattern = re.compile("^([A-Za-z &])*$")
current_time = datetime.now()
def parse_reviews(c):
    return literal_eval(c.replace('nan',"''"))
def parse_cusine(c):
    try:        
        return literal_eval(c.replace("NaN", "[]"))
    except Exception:
        if literal_string_pattern.match(c):
            return "['{}']".format(c)
        else:
            raise Exception('Failed to parse cusine: ' + c)
            
def review_dates(reviews):        
    if len(reviews) > 1:
        return list(map(lambda d: datetime.strptime(d,'%m/%d/%Y'), reviews[1]))
    else:
        return []

def diff_in_days(d):
    if len(d) > 1:
        return abs((d[1] - d[0]).days)
    else:
        return -1
    
def review_freshness(rd):
    if len(rd) > 1:
        return min(list(map(lambda d: abs((current_time - d).days), rd)))
    else:
        return -1
    
def price_level(p):
    if p == '$':
        return 1
    elif p == '$$ - $$$':
        return 2
    elif p == '$$$$':
        return 3
    else:
        return -1
def extract_name(row):
    url = row['URL_TA']
    reviews = 'Reviews'
    start = url.find(reviews) + len(reviews) + 1
    return url[start:].split('-')[0]

Преобразуем строчные признаки Cuisine Style и Review Dates в списки, применим lable encoding, добавим новые признаки:

In [ ]:
data['Reviews'] = data.Reviews.apply(parse_reviews)
data['Cuisine Style'] = data['Cuisine Style'].apply(parse_cusine)
data['cusine_count'] = data['Cuisine Style'].apply(lambda c: len(c))
data['price_level'] = data['Price Range'].apply(price_level)
data['Review Dates'] = data.Reviews.apply(review_dates)
data['days_between_reviews'] = data['Review Dates'].apply(diff_in_days)
data['review_freshness'] = data['Review Dates'].apply(review_freshness)
restraunts_per_city = data.groupby('City').ID_TA.count().reset_index()
restraunts_per_city.columns = ['City', 'restraunts_per_city']
data = data.merge(restraunts_per_city, on='City')
data['restaurant_name'] = data.apply(extract_name, axis=1)
restaurants_in_chain = data[['restaurant_name','ID_TA']].drop_duplicates()['restaurant_name'].value_counts().reset_index()
restaurants_in_chain.columns = ['restaurant_name', 'restaurants_in_chain']
data = data.merge(restaurants_in_chain, on=['restaurant_name'])
data['is_chain_restaurant'] = data['restaurants_in_chain'].apply(lambda r: 1 if r > 1 else 0)
data.drop(columns=['restaurants_in_chain'], inplace=True)
restaurants_in_chain_per_city = data[['restaurant_name','ID_TA', 'City']].drop_duplicates().groupby(['City', 'restaurant_name']).ID_TA.count().reset_index()
restaurants_in_chain_per_city.columns = ['City', 'restaurant_name', 'restaurants_in_chain']
data = data.merge(restaurants_in_chain_per_city, on=['City', 'restaurant_name'])

cusine_count - кол-во кухонь, представленных в ресторане 

days_between_reviews - кол-во дней между опубликованными отзывами

review_freshness - колво дней от самого свежего отзыва до сегодня

restraunts_per_city - общее кол-во ресторанов в городе

restaurants_in_chain - кол-во точек ресторанной сети в городе

is_chain_restaurant - является ли ресторан сетевым

Посмотрим на распределение признака Number of Reviews:

In [ ]:
_, ax = plt.subplots(figsize=(20,3))
ax.set_yscale('log')
data[data['sample'] == 1]['Number of Reviews'].hist(bins=500, ax=ax)

Посмотрим на распределение признака price_level

In [ ]:
data[data['sample'] == 1]['price_level'].value_counts()

Видно, что значительная часть ресторанов не отнесена к тому или иному ценовому сегменту.
Посмотрим, как пересекаются группы без отзывов и ценовой классификации:

In [ ]:
data.fillna(-1).loc[(data['sample'] == 1) & (data['Number of Reviews'] <= 0) & (data['price_level'] < 1)].ID_TA.count()

К "неизведанным" относится небольшая доля ресторанов - эффективный признак по этим данным собрать не получиться

Исправим признак Number of Reviews с помощью информации об опубликованных отзывах:

In [ ]:
def actual_number_of_reviews(reviews):
    return len(list(filter(lambda r: len(r) > 0, reviews)))

def fix_number_of_reviews(row):
    actual = actual_number_of_reviews(row['Reviews'])
    given = row['Number of Reviews']
    return actual if given < actual else given

data['Number of Reviews'] = data[['Number of Reviews','Reviews']].fillna(0).apply(fix_number_of_reviews, axis=1)

Заполним пропуски price_level медианными значениями по городам:

In [ ]:
median_price_by_city = data.query('(sample == 1) and (price_level > 0)').groupby('City').price_level.median().reset_index()
median_price_by_city.columns = ['City', 'price_level_median']
median_price_ranges = { m['City']:m['price_level_median'] for m in median_price_by_city.to_dict('rows') }
def fix_price_level(row):
    review_num = row['Number of Reviews']
    given = row['price_level']
    if (review_num == 0) and (given < 0):
        return 0
    elif given < 0: 
        return median_price_ranges[row['City']]
    else:
        return given
data['price_level'] = data.apply(fix_price_level, axis=1)

Посмотрим на распределение признака Ranking (выглядит смещенным)

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
data[data['sample'] == 1]['Ranking'].hist(bins=100)

Посмотрим на распределение признака Ranking по каждому из возможных значений Rating:

In [ ]:
ratings = sorted(data[data['sample'] == 1]['Rating'].unique())
rows = round(len(ratings) / 2) + 1
ncols = 2
fig = plt.figure(figsize=(20, 20))

for i, r in enumerate(ratings):
    ax = fig.add_subplot(rows, ncols, i + 1)
    series = data.query("(sample == 1) and (Rating == {})".format(r))['Ranking']
    series.hist(bins=100)
    ax.set_xlabel("Rating == {}".format(r))
    

Видна зависимость Ranking ресторана и его рейтинга, однако, для средне-негативных оценок (2-2.5) она не столь очевидна, как для высоких или обсолютно-низких.
Попробуем нормализовать признак Ranking, добавим balanced_ranking - Ranking, взвешенный на кол-во ретсоранов в городе:

In [ ]:
data['balanced_ranking'] = data.Ranking / data.restraunts_per_city

Сравним распределение признаков Ranking и balanced_ranking

In [ ]:
plt.rcParams['figure.figsize'] = (20,5)
data[data['sample'] == 1][['Ranking','balanced_ranking']].hist(bins=100)

Посмотрим на распределение времени между отзывами:

In [ ]:
actual_review_delays = data.query('(sample == 1) and (days_between_reviews >= 0)')['days_between_reviews']
_, ax = plt.subplots(figsize=(20,3))
ax.set_yscale('log')
plt.xticks(np.arange(actual_review_delays.min(), actual_review_delays.max() + 1, 100.0))
actual_review_delays.hist(bins=500, ax=ax)

есть подозрение, что значения более ~1800 получены в результате ошибок ввода / парсинга, попробуем их отсеять, отсутствующие данные заполним медианой

признак review_freshness так же починим с помощью медианы

In [ ]:
days_between_reviews_median = data.query('(sample == 1) and (days_between_reviews >= 0)').days_between_reviews.median()
data['days_between_reviews'] = data['days_between_reviews'].apply(lambda d: d if (d < 1800) & (d >= 0) else days_between_reviews_median)
review_freshness_median = data.query('(sample == 1) and (review_freshness >= 0)').review_freshness.median()
data['review_freshness'] = data.review_freshness.apply(lambda d: d if d >=0 else review_freshness_median)

Добавляем признак "Страна":

In [ ]:
city_to_country = {'Paris': 'France',
 'Stockholm': 'Sweden',
 'London': 'United Kingdom',
 'Berlin': 'Germany',
 'Munich': 'Germany',
 'Milan': 'Italy',
 'Bratislava': 'Slovakia',
 'Vienna': 'Austria',
 'Rome': 'Italy',
 'Barcelona': 'Spain',
 'Madrid': 'Spain',
 'Dublin': 'Ireland',
 'Brussels': 'Belgium',
 'Zurich': 'Switzerland',
 'Warsaw': 'Poland',
 'Budapest': 'Hungary',
 'Copenhagen': 'Denmark',
 'Amsterdam': 'Netherlands',
 'Lyon': 'France',
 'Hamburg': 'Germany',
 'Lisbon': 'Portugal',
 'Porto': 'Portugal',
 'Prague': 'Czech Republic (Czechia)',
 'Oslo': 'Norway',
 'Helsinki': 'Finland',
 'Edinburgh': 'United Kingdom',
 'Geneva': 'Switzerland',
 'Ljubljana': 'Slovenia',
 'Athens': 'Greece',
 'Luxembourg': 'Luxembourg',
 'Krakow': 'Poland'}
data['Country'] = data.City.apply(lambda c: city_to_country[c])


Добавляем размер городского населения по странам к data-frame'у:

In [ ]:
data = data.merge(population_data, on='Country', how='left')

Добавим категориальные факторы кухонь по найциональным признакам (американская, восточно-европейская, кавказская и т.д.), а так же по типам заведения:

In [ ]:
cuisine_origin_groups = {
    'european': ['Italian','French','Spanish','Swiss','Belgian','Dutch','Austrian','Czech','Greek','German','Portuguese','Central European','Spanish','Mediterranean', 'European', 'Pizza'],
    'uk': ['British','Scottish','Welsh','Irish'],
    'scandinavian': ['Scandinavian','Norwegian','Balti','Swedish','Danish'],
    'east_europe':['Russian','Latvian','Romanian','Croatian','Slovenian','Albanian','Ukrainian','Hungarian','Polish','Eastern European'],
    'asian': ['Fujian','Central Asian','Taiwanese','Tibetan','Korean','Vietnamese','Thai','Asian','Singaporean', 'Indonesian', 'Mongolian', 'Uzbek', 'Yunnan','Xinjiang','Minority Chinese','Chinese'],
    'india':['Vegetarian Friendly', 'Indian', 'Sri Lankan', 'Nepali'],
    'japanese':['Sushi', 'Japanese'],
    'international': ['International', 'Fusion'],
    'colonial': ['Native American', 'American', 'Central American', 'Cajun & Creole', 'Canadian', 'Hawaiian','Australian','Jamaican', 'New Zealand', 'Polynesian'],
    'mideast_and_africa': ['Middle Eastern', 'Kosher', 'Israeli', 'Lebanese', 'Moroccan', 'Halal', 'African', 'Turkish', 'Tunisian', 'Egyptian', 'Arabic', 'Ethiopian', 'Afghani'],
    'caucasus': ['Armenian', 'Caucasian', 'Georgian', 'Azerbaijani'],
    'latam': ['Ecuadorean','Chilean','Colombian','Venezuelan','Peruvian','Brazilian','Argentinean','Latin', 'South American', 'Mexican', 'Salvadoran']
}
cuisine_specialization = {
    'drinks': ['Brew Pub','Wine Bar','Pub','Bar'],
    'general': ['Fast Food', 'Street Food', 'Cafe', 'Grill','Barbecue', 'Pizza', 'Diner', 'Soups', 'Fusion'],
    'specialty':['Gastropub','Delicatessen','Seafood', 'Sushi', 'Steakhouse'],
    'lifestyle': ['Vegetarian Friendly','Vegan Options','Gluten Free Options','Healthy', 'Halal', 'Contemporary']
}
regional_cusines = {'european', 'asian', 'american', 'colonial', 'mideast_and_africa', 'latam', 'caucasus'}
local_cuisines = {'France': ['French'],
 'Sweden': ['Swedish','Scandinavian'],
 'United Kingdom': ['British', 'Scottish', 'Welsh'],
 'Germany': ['German', 'Dutch'],
 'Portugal': ['Portuguese'],
 'Italy': ['Italian', 'Pizza'],
 'Slovakia': [],
 'Austria': ['Dutch','Austrian'],
 'Italy': ['Italian', 'Pizza'],
 'Spain': ['Spanish'],
 'Ireland': ['Irish'],
 'Belgium': ['Belgian'],
 'Switzerland':['Swiss'],
 'Poland': ['Polish','Eastern European'],
 'Hungary': ['Hungarian','Eastern European'],
 'Denmark': ['Danish','Scandinavian'],
 'Netherlands': [],
 'Czech Republic (Czechia)': ['Czech'],
 'Norway': ['Scandinavian','Norwegian'],
 'Finland': ['Scandinavian'],
 'Slovenia': ['Slovenian'],
 'Greece': ['Greek'],
 'Luxembourg': []
}
def cuisine_origin(cs):
    found_types = {}
    if 'international' in cs:
        return 'international'
    for cg, cuisines in cuisine_origin_groups.items():
        for c in cs:
            if c in cuisines:
                found_types[cg] = found_types.get(cg, 0) + 1
    if len(found_types) == 0:
        return 'no_origin'                
    if len(found_types.keys() & regional_cusines) > 1:
        return 'international'
    else:
        return sorted(found_types.items(), key=lambda item: item[1])[-1][0]
def cuisine_spec(cs):
    found_types = {}
    for cg, cuisines in cuisine_specialization.items():
        for c in cs:
            if c in cuisines:
                found_types[cg] = found_types.get(cg, 0) + 1
    if len(found_types) == 0:
        return 'no_spec'
    return sorted(found_types.items(), key=lambda item: item[1])[-1][0]
def is_local_cuisine(row):
    locs = local_cuisines[row['Country']]
    return 1 if set(locs) & set(row['Cuisine Style']) else 0
data['cuisine_origin'] = data["Cuisine Style"].apply(cuisine_origin)
data['cuisine_spec'] = data["Cuisine Style"].apply(cuisine_spec)
data['is_local_cuisine'] = data.apply(is_local_cuisine,axis=1)

cuisine_origin - национальный признак кухни

cuisine_spec - специализация кухни (drinks, general, specialty..)

is_local_cuisine - является ли кухня "местной" (например: францускую кухню в Париже считаем местной, китайскую кухню в Риме местной не считаем)

Проверим, различаются ли Rating в группах полученных категориальных факторов:

In [ ]:
from scipy.stats import kruskal
factors_to_check = {
    'cuisine_origin': data[data['sample'] == 1].cuisine_origin.unique(),
    'cuisine_spec': data[data['sample'] == 1].cuisine_spec.unique(),
    'is_local_cuisine': data[data['sample'] == 1].is_local_cuisine.unique(),
    'is_chain_restaurant': data[data['sample'] == 1].is_chain_restaurant.unique()
}

for c, values in factors_to_check.items():
    _, p = kruskal(*[data[data[c] == v]['Rating'] for v in values])
    if p < 0.05:
        print("{} seems significant! p-value: {}".format(c, p))
    else:
        print("{} seems irrelevant".format(c, p))

Судя по всему, мой самый любимый фактор локальности кухни (is_local_cuisine) не подходит для оценки рейтинга :(, мы не будем его исопльзовать.

In [ ]:
data.drop(columns=['is_local_cuisine'],inplace=True)

Применим one-hot-encoding к категориальным факторам cuisine_origin, cuisine_spec, City:

In [ ]:
data = pd.get_dummies(data, columns=['cuisine_origin','cuisine_spec'])

In [ ]:
data['City_is'] = data['City']
data = pd.get_dummies(data, columns=['City_is'], dummy_na=True)

In [ ]:
from itertools import combinations

def get_corr(dataframe, name):
    digit_columns = dataframe.select_dtypes(include=['int64', 'float64']).columns
    combinations_all = list(combinations(digit_columns, 2))
    corr = {}
    
    for c in combinations_all:
        col1 = dataframe[c[0]]
        col2 = dataframe[c[1]]
        corr[c] = col1.corr(other=col2)

    idx = pd.MultiIndex.from_tuples(corr.keys(), names=['A', 'B'])
    #corr = pd.Series(corr, index = idx).sort_values()
    corr = pd.DataFrame(list(corr.values()), index=idx,
                        columns=[name]).sort_values(by=name)
    return corr

corr = get_corr(data.drop(['sample'], axis=1), 'general')

In [ ]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.width', 300)
pd.set_option('display.max_colwidth', 300)

perc25 = (abs(corr.max()) + abs(corr.min())) * \
    0.125  # ((abs(a) + abs(b)) / 2) * 0.25
display(corr[(corr < -perc25) | (corr > perc25)].dropna())

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    df_output.drop(['Restaurant_id','ID_TA', 'Ranking'], axis = 1, inplace=True)    

    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

#### Запускаем и проверяем что получилось

In [ ]:
data.columns

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
y_pred

Поскольку рейтинг кратен 0.5, округляем

In [ ]:
def round_rating_pred(rating_pred):
    if rating_pred <= 0.5:
        return 0.0
    if rating_pred <= 1.5:
        return 1.0
    if rating_pred <= 1.75:
        return 1.5
    if rating_pred <= 2.25:
        return 2.0
    if rating_pred <= 2.75:
        return 2.5
    if rating_pred <= 3.25:
        return 3.0
    if rating_pred <= 3.75:
        return 3.5
    if rating_pred <= 4.25:
        return 4.0
    if rating_pred <= 4.75:
        return 4.5
    return 5.0

In [ ]:
for i in range(len(y_pred)):
    y_pred[i] = round_rating_pred(y_pred[i])

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических. 
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_df_sorted = data.query('sample == 0').sort_values(by='Restaurant_id')
actual_y = test_df_sorted.Rating.values  
test_data = preproc_data(test_df_sorted.drop(['sample','Rating'], axis=1))
test_data.sample(10)

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
for i in range(len(predict_submission)):
    predict_submission[i] = round_rating_pred(predict_submission[i])

In [ ]:
predict_submission

In [ ]:
result = pd.DataFrame(data = {'Restaurant_id':test_df_sorted['Restaurant_id'].values, 'Rating': predict_submission})
submission = sample_submission.drop(columns=['Rating'])
submission = submission.merge(result, on='Restaurant_id')
submission.to_csv('submission.csv', index=False)
submission.head(10)